In [17]:
import os
root='/content/drive/MyDrive/ComputerVision/ObjectDetection/LeakageDetection'
print(os.getcwd())
os.chdir(root)
print(os.getcwd())

!rm '/content/drive/MyDrive/ComputerVision/ObjectDetection/LeakageDetection/data/images/.DS_Store' 

/content/drive/MyDrive/ComputerVision/ObjectDetection/LeakageDetection
/content/drive/MyDrive/ComputerVision/ObjectDetection/LeakageDetection
rm: cannot remove '/content/drive/MyDrive/ComputerVision/ObjectDetection/LeakageDetection/data/images/.DS_Store': No such file or directory


In [18]:
# image
from PIL import Image
import cv2
import numpy as np

# pytorch
import torch
import torch.utils.data
import utils
import transforms as T
from engine import train_one_epoch, evaluate
from google.colab.patches import cv2_imshow
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [19]:
# Specify a path
PATH = "leakage_detection_model_1681583049.9375646.pt"
# Load
model = torch.load(PATH)
model.eval()

from torchvision import transforms

def perform_inference(image):
    # transform the input image to tensor
    test_transforms = transforms.Compose([transforms.ToTensor() ])
    img = test_transforms(image)
    # inference
    with torch.no_grad():
        prediction=model([img.to(device)])
    # print(prediction)
    # get results
    box = list(prediction[0]['boxes'].cpu().numpy()[0])
    score = round(prediction[0]['scores'].cpu().numpy()[0], 2)
    bbox = [int(x) for x in box]
    imag=img.mul(255).permute(1, 2, 0).byte().numpy()

    imag = np.ascontiguousarray(imag, dtype=np.uint8)
    # mark the bboxes
    start_point=(bbox[0],bbox[1])
    end_point=(bbox[2],bbox[3])
    
    # if score:
    image = cv2.putText(img=imag, 
                        text=f'leakage: {str(score)}' , 
                        org=(500, 100), 
                        fontFace = cv2.FONT_HERSHEY_SIMPLEX,  
                        fontScale=0.5, 
                        color=(255, 255, 255), 
                        thickness=2)
    image = cv2.rectangle(image, start_point, end_point, color=(0,255,255), thickness=2)
    # cv2.imshow('Frame',image)
    # cv2.waitKey(0)
    return image

In [25]:
# load the video
cap=cv2.VideoCapture('sink.mp4')


# Properties
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video Writer 
video_writer = cv2.VideoWriter('sink_output.mp4', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width,height))
                               
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    image=perform_inference(frame)
    # Write out frame 
    video_writer.write(image)
    # Display the resulting frame
    # cv2_imshow(image)
 
    # # Press Q on keyboard to  exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
cap.release()
cv2.destroyAllWindows()

# Release video writer
video_writer.release()